<a href="https://colab.research.google.com/github/najimamuhammad/Programs/blob/main/Churn_Modelling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import math
from tensorflow import keras
import matplotlib.pyplot as plt
from tensorflow.keras import regularizers


In [ ]:
df=pd.read_csv('Churn_Modelling.csv')

In [ ]:
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [ ]:
df.drop(['RowNumber','CustomerId','Surname'],axis=1,inplace=True)

In [ ]:
df.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [ ]:
df_new=pd.get_dummies(df,['Geography','Gender'],drop_first=True)

In [ ]:
df_new.head()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_Germany,Geography_Spain,Gender_Male
0,619,42,2,0.00,1,1,1,101348.88,1,False,False,False
1,608,41,1,83807.86,1,0,1,112542.58,0,False,True,False
2,502,42,8,159660.80,3,1,0,113931.57,1,False,False,False
3,699,39,1,0.00,2,0,0,93826.63,0,False,False,False
4,850,43,2,125510.82,1,1,1,79084.10,0,False,True,False


In [ ]:
df_new.shape

(10000, 12)

In [ ]:
df_new.columns.difference(['Exited'])

Index(['Age', 'Balance', 'CreditScore', 'EstimatedSalary', 'Gender_Male',
       'Geography_Germany', 'Geography_Spain', 'HasCrCard', 'IsActiveMember',
       'NumOfProducts', 'Tenure'],
      dtype='object')

In [ ]:
X=df_new[df_new.columns.difference(['Exited'])]

In [ ]:
X.head()

,Age,Balance,CreditScore,EstimatedSalary,Gender_Male,Geography_Germany,Geography_Spain,HasCrCard,IsActiveMember,NumOfProducts,Tenure
0,42,0.00,619,101348.88,False,False,False,1,1,1,2
1,41,83807.86,608,112542.58,False,False,True,0,1,1,1
2,42,159660.80,502,113931.57,False,False,False,1,0,3,8
3,39,0.00,699,93826.63,False,False,False,0,0,2,1
4,43,125510.82,850,79084.10,False,False,True,1,1,1,2


In [ ]:
y=df_new['Exited']

In [ ]:
y.head()

0    1
1    0
2    1
3    0
4    0
Name: Exited, dtype: int64

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
train_x, test_x, train_y, test_y = train_test_split(X,y,test_size=0.3,random_state=123)

In [ ]:
train_x.shape

(7000, 11)

In [ ]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
train_X=sc.fit_transform(train_x)
test_X=sc.fit_transform(test_x)

In [ ]:
input=keras.Input(shape=(train_x.shape[-1],))
x1=keras.layers.Dense(6, activation='relu')(input)
x2=keras.layers.Dense(6,activation='relu')(x1)
output=keras.layers.Dense(1,activation='sigmoid')(x2)


In [ ]:
model=keras.Model(input,output,name='churn_model')

In [ ]:
initial_learning_rate=0.01
def lr_exp_decay(epoch,lr):
  k=0.1
  return initial_learning_rate*math.exp(-k*epoch)


In [ ]:
lr_schedule = keras.callbacks.LearningRateScheduler(lr_exp_decay,verbose=1)

In [ ]:
sgd=keras.optimizers.SGD(learning_rate=initial_learning_rate)


In [ ]:
model.compile(optimizer=sgd,loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:
model.summary()

Model: "churn_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 11)]              0         
                                                                 
 dense_6 (Dense)             (None, 6)                 72        
                                                                 
 dense_7 (Dense)             (None, 6)                 42        
                                                                 
 dense_8 (Dense)             (None, 1)                 7         
                                                                 
Total params: 121 (484.00 Byte)
Trainable params: 121 (484.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
checkpointbest=keras.callbacks.ModelCheckpoint(filepath='weights.hdf5',verbose=1,save_best_only=True)
model.fit(train_X,train_y,epochs=20,validation_data=(test_X,test_y),callbacks=[checkpointbest,lr_schedule])


Epoch 1: LearningRateScheduler setting learning rate to 0.01.
Epoch 1/20
219/219 [==============================] - ETA: 0s - loss: 0.6816 - accuracy: 0.6090
Epoch 1: val_loss improved from inf to 0.57932, saving model to weights.hdf5
219/219 [==============================] - 3s 5ms/step - loss: 0.6816 - accuracy: 0.6090 - val_loss: 0.5793 - val_accuracy: 0.8013 - lr: 0.0100

Epoch 2: LearningRateScheduler setting learning rate to 0.009048374180359595.
Epoch 2/20
 49/219 [=====>........................] - ETA: 0s - loss: 0.5729 - accuracy: 0.7978

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


217/219 [============================>.] - ETA: 0s - loss: 0.5508 - accuracy: 0.7967
Epoch 2: val_loss improved from 0.57932 to 0.52995, saving model to weights.hdf5
219/219 [==============================] - 1s 4ms/step - loss: 0.5506 - accuracy: 0.7967 - val_loss: 0.5300 - val_accuracy: 0.7990 - lr: 0.0090

Epoch 3: LearningRateScheduler setting learning rate to 0.008187307530779819.
Epoch 3/20
205/219 [===========================>..] - ETA: 0s - loss: 0.5230 - accuracy: 0.7951
Epoch 3: val_loss improved from 0.52995 to 0.51344, saving model to weights.hdf5
219/219 [==============================] - 1s 4ms/step - loss: 0.5219 - accuracy: 0.7959 - val_loss: 0.5134 - val_accuracy: 0.7983 - lr: 0.0082

Epoch 4: LearningRateScheduler setting learning rate to 0.007408182206817179.
Epoch 4/20
208/219 [===========================>..] - ETA: 0s - loss: 0.5114 - accuracy: 0.7954
Epoch 4: val_loss improved from 0.51344 to 0.50600, saving model to weights.hdf5
219/219 [=========================

In [ ]:
y_pred=model.predict(test_X)

94/94 [==============================] - 0s 2ms/step


In [ ]:
import sklearn.metrics as metrics

In [ ]:
metrics.roc_auc_score(test_y,y_pred)

0.6349112303524905

In [ ]:
y_pred=(y_pred>0.2)

In [ ]:
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(test_y,y_pred)

In [ ]:
print(cm)

[[1542  853]
 [ 266  339]]


In [ ]:
print(metrics.classification_report(test_y, y_pred))

              precision    recall  f1-score   support

           0       0.85      0.64      0.73      2395
           1       0.28      0.56      0.38       605

    accuracy                           0.63      3000
   macro avg       0.57      0.60      0.56      3000
weighted avg       0.74      0.63      0.66      3000

